In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import json

# 웹 드라이버 설정 (Chrome 사용)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 백그라운드 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# 네이버 웹툰 페이지 이동
url = "https://comic.naver.com/webtoon"
driver.get(url)

time.sleep(3)  # 페이지 로드 대기

# 스크롤 끝까지 내리기
scroll_pause_time = 2  # 페이지 로딩 대기 시간
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# HTML 내용을 파일로 저장
html_filename = "naver_webtoon.html"
with open(html_filename, "w", encoding="utf-8") as file:
    file.write(driver.page_source)

driver.quit()

print(f"웹툰 페이지 HTML이 {html_filename} 파일로 저장되었습니다.")

# 저장된 HTML 파일에서 데이터 추출
from bs4 import BeautifulSoup

with open(html_filename, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# 웹툰 정보를 담을 딕셔너리
webtoon_data = {}

# 요일별 웹툰 리스트 추출
weekday_elements = soup.find_all(class_="WeekdayMainView__daily_all_item--DnTAH")

for weekday in weekday_elements:
    webtoons = weekday.find_all(class_="DailyListItem__item--LP6_T")
    for webtoon in webtoons:
        try:
            image_element = webtoon.find("img", class_="Poster__image--d9XTI")
            if image_element:
                webtoon_name = image_element["alt"]
                img_src = image_element["src"]
                webtoon_id = img_src.split("/")[-3]  # URL에서 ID 추출
                webtoon_data[webtoon_name] = {"id": int(webtoon_id)}
        except Exception as e:
            print(f"Error processing webtoon: {e}")

# JSON 파일로 저장
output_filename = "webtoon_info.json"
with open(output_filename, "w", encoding="utf-8") as json_file:
    json.dump(webtoon_data, json_file, ensure_ascii=False, indent=4)

print(f"웹툰 정보가 {output_filename} 파일로 저장되었습니다.")

웹툰 페이지 HTML이 naver_webtoon.html 파일로 저장되었습니다.
웹툰 정보가 webtoon_info.json 파일로 저장되었습니다.
